# Preprocessing

* Veri keşfi
* Tekrarlı kayıları giderelim
* Null kayıtlardan kurtulalım.
* Temel istatistiklere bakalım.
* Tahmin için kullanılacak kolonları seçelim.
* Koralasyonu inceleyelim.

In [1]:
import numpy as np
import pandas as pd

## CSV dosyasını okuyalım

In [2]:
df = pd.read_csv("./Raw_Meşrubat_Satışları.csv")

df.head()
df.columns
df.shape
df.dtypes #df.info

print("{} satır {} kolon".format(*df.shape)) #squence unpacking

32 satır 7 kolon


## Tekrarlı kayıtları temizleyelim

In [3]:
df.nunique() #benzersiz kayırların adedi
df.duplicated() #tekrar edenler Her satır için T/F döner
df.duplicated().any()

#
df[df.duplicated()] #tekrar eden satır.

#
df.query("Tarih == '2019-07-06'")

#drop edelim
df = df.drop_duplicates() #orjinalini etkilemez

#kolon bazlı tekrarlı var mı?
df.duplicated(subset=["Tarih"]).any()

False

## NULL değerleri giderelim

In [4]:
df.isna().any() #hangi kolonda null değer var
df.isna().sum() #hangi kolonda kaç adet null değer var

#
'''
#grafikle görebilirsin:
import missingno
#missingno.matrix(df)
'''

df.query("Tarih.isna()",engine='python') #.index ile sıra numarası alınabilir.
#Burada faydalı bilgiler var. drop diyemeyiz.

,Tarih,Lokasyon,Limon,Portakal,Sıcaklık,Broşür,Fiyat
8,NaN,Plaj,86,82,28,113.0,5.0


In [5]:
# Tarihin ne olabileceğine bakalım
#df.Tarih#.sort_values(ascending=True)#.iloc[0:10] #NaN en sona atanıyor.
df.Tarih.sort_index().head(10) #8. indexte tarih boş. muhtemel tarih 2019-07-08

#2019-07-08 böyle bir tarih varmı? 
df.query("Tarih == '2019-07-08'").__len__() #satır gelmedi.

0

### Null değerleri dolduralım
* Tarihsel alanda 2019-07-08 tarihini verelim
* Sayısal olan Broşür alanında o alanın ortalama değerini atayalım.

In [6]:
#null değerleri silmek için
#df.dropna(axis=0, how='any') #varsayılan değerledir. herhanfi bir null varsa satırı siler.

#null değerleri doldurmak için
df.fillna({
            "Tarih":"2019-07-08",
            "Broşür":df.Broşür.mean(skipna=True) #varsayılan NULL değer görmezden gelinir.
          }
          ,inplace=True) #orjinalini etkilemez.

#
df.isnull().any().any() #tabloda hiç null var mı?

False

## Tahmin için gerekli kolonları oluşturalım
* Tarih kolonu yerine gün adını alalım.
* Limon ve Portakal satışlarını toplayıp Satış kolonu üretelim.
* Çıkarım yapmak için isteyeceğimiz kolonların dışında kalanları kaldıralım

In [7]:
df.dtypes #tarih kolonu datetime değil. astype ile çevirebiliriz. fakat bunun yerine pd.to_datetime kullanacağız.

#1. yöntem kolon ekleme
df["Gün"] = pd.to_datetime(df.Tarih).dt.strftime("%A") #dt modulu
#tahminlemede gün adını isteyeceğiz haftanın gün sırası lazım değil
df["HaftanınGünü"] = pd.to_datetime(df.Tarih).dt.dayofweek #0 pazartesidir. 

#2. yöntem kolon ekleme
df.eval("Satış = Limon + Portakal", 
        inplace=True) #virgül koymadan alt satıra geçerek yeni kolonar üretelebilir.


#Çıkarım yapacağımız kolonlar kalsın diğerlerini kaldıralım.
df.drop(columns=["Tarih","Limon","Portakal","HaftanınGünü"],
        inplace=True)

df.head(3)


,Lokasyon,Sıcaklık,Broşür,Fiyat,Gün,Satış
0,Park,22,90.0,2.5,Monday,137
1,Park,23,90.0,5.0,Tuesday,139
2,Park,22,104.0,5.0,Wednesday,148


### Kolonları sıralanmasını değiştirelim.

In [8]:
#df.columns.tolist()
yeniKolonSirasi = ['Gün','Lokasyon', 'Sıcaklık', 'Broşür', 'Fiyat', 'Satış']
df = df[yeniKolonSirasi]

df.head(1)

,Gün,Lokasyon,Sıcaklık,Broşür,Fiyat,Satış
0,Monday,Park,22,90.0,2.5,137


## Temel İstatistiklere Bakalım

In [9]:
df.describe().T

#
#önceki versiyonda .style.highlight_between(left=0.3, right = 0.99, color='green') kullanılırdı.
df.corr().style.applymap(lambda v: 'background-color: green;color: white;' if v>0.2 else None)\
               .applymap(lambda v: 'background-color: red;color: white;' if v<-0.2 else None)\
               .applymap(lambda v: 'opacity: 20%;' if v==1 else None)             

,Sıcaklık,Broşür,Fiyat,Satış
Sıcaklık,1.000000,0.314423,0.030857,0.604631
Broşür,0.314423,1.000000,0.064502,0.801649
Fiyat,0.030857,0.064502,1.000000,-0.248873
Satış,0.604631,0.801649,-0.248873,1.000000


## Görselleştirme

In [ ]:
#DataFrame üzerinden plot ile
df.plot.scatter("Sıcaklık","Satış")

#
#df.boxplot(column="Satış",by="Lokasyon")
df.boxplot(column="Satış",by="Gün")

In [ ]:
#Grafik çizimini bir fonksiyona bağlayalım.
def dagilimGrafikleriCiz(x,y="Satış"):
    plt.scatter(df[x], df[y])
    
    #----------
    w,b = np.polyfit(df[x], df[y], deg=1)
    f = lambda X:w*X + b

    plt.plot(df[x],f(df[x]),color='red')
    #----------
    plt.show()

#-------
dagilimGrafikleriCiz("Sıcaklık","Satış")


In [ ]:
plt.style.use("seaborn") #plt.style.available

#
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(16,6), sharex=True)

axs[0].hist(df.Broşür)
axs[1].boxplot(df.Broşür, vert=False, showmeans=True)

#axs[1].axis("off")
axs[1].get_yaxis().set_visible(False)

plt.text(x=df.Broşür.mean(), y=1, s=f" {df.Broşür.mean():,.2f}")

plt.show()

### Eğitim için gerekli verileri kaydedelim.

In [13]:
import os

#klasor oluştur
os.makedirs("../Training",exist_ok=True)

#o klasore kaydet
df.to_csv("../Training/dataForTraining.csv",index=False)

#var mı kontrol et
os.path.exists("../Training/dataForTraining.csv")

True